In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib import rcParams
%matplotlib inline

 # Data Understanding

In [ ]:
# Read Train/Test  Data
train=pd.read_csv("../input/health-insurance-cross-sell-prediction/train.csv")
test=pd.read_csv("../input/health-insurance-cross-sell-prediction/test.csv")

In [ ]:
train.head()

In [ ]:
#Check data shape
print('There are {} rows in the training data and {} varaible columns on training data '.format
      (train.shape[0],train.shape[1]))

# check how many unique customer id's are there
print('There are also {} unique customers. That is there are no duplication info on customer'.format(len(train['id'].unique())))

In [ ]:
print(train.columns)

# Exploratory Analysis of data
### Lets get feel of each variable

In [ ]:
# 1. Check data types
print(train.dtypes)

In [ ]:

# 2. Let's check are there any missing values across each variable
print(train.isnull().sum())

#### There aren't any  missing values in  variable columns

In [ ]:
# 3.Univarte Analysis

# 3.1 Distribution of Response
print(train.Response.unique())


In [ ]:
# Returns the sum of all not-null values in `Response` column
n_response = train['Response'].value_counts().sum()
print('There are {} responses'.format(n_response))


# Count the frequency of unique values in the `Response` column of train dataframe. 
# By default, returns the decreasing order of the frequency.
response_counts = train['Response'].value_counts()
print(response_counts)


In [ ]:
### Ploting Functions

def single_categorical_prop_plot(data,variable,order_=True,lable_rotation=0):
    # Specify the figure size in inches, for both X, and Y axes
    rcParams['figure.figsize'] = 5,4
    # Considering the same chart from the Example 1 above, print the text (proportion) BELOW the bars
    base_color = sb.color_palette()[1]
    
    type_counts = data[variable].value_counts()
    
    # Returns the sum of all not-null values in `Response` column
    n_response = data[variable].value_counts().sum()
    
    
    if order_:
        type_order = type_counts.index
        sb.countplot(data=data, x=variable, color=base_color, order=type_order)
    else:
        sb.countplot(data=data, x=variable, color=base_color)
    
    
    
    # get the current tick locations and labels
    locs, labels = plt.xticks(rotation=lable_rotation) 

    # loop through each pair of locations and labels
    for loc, label in zip(locs, labels):
        #print(loc)
        #print(label)
        # get the text property for the label to get the correct count
        if isinstance(type_counts.index.values[0],str):
            count = type_counts[(label.get_text())]
        else:
            count = type_counts[float(label.get_text())]
        pct_string = '{:0.1f}%'.format(100*count/n_response)

        # print the annotation just below the top of the bar
        plt.text( loc,count,pct_string, ha = 'center', color = 'black')
    
        
    plt.xlabel('{} '.format(variable))  
    plt.title('{} distribution plot'.format(variable))

In [ ]:
# Let's check response variable distribution

single_categorical_prop_plot(data=train,variable='Response')
total_string='Total_Responses:{}'.format(train.shape[0])
plt.text(1,300000,total_string,ha = 'center', color = 'black')


From the above plot it seems that our response rate is imbalanced.So we need to take care while building model

In [ ]:
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [11, 5]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(1, 2, 1) 
single_categorical_prop_plot(data=train,variable='Gender')
plt.title('Train Gender Distribution')

# histogram on right, example of too-small bin size
plt.subplot(1, 2, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Gender')
plt.title('Test Gender Distribution')

Clearly it shows distribution of Gender in both train and test dataset is same.Also Male distribtion in the dataset is slightly higher than Female Distribution

In [ ]:
#Lets examine distribution of driving license possesion 
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [11, 5]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(1, 2, 1) 
single_categorical_prop_plot(data=train,variable='Driving_License',order_=True)
plt.title('Train Driving_License Distribution')

# histogram on right, example of too-small bin size
plt.subplot(1, 2, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Driving_License',order_=True)
plt.title('Test Driving_License Distribution')

In [ ]:
#Lets examine distribution of driving license possesion 
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [11, 5]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(1, 2, 1) 
single_categorical_prop_plot(data=train,variable='Vehicle_Age',order_=True)
plt.title('Train Vehicle_Age Distribution')

# histogram on right, example of too-small bin size
plt.subplot(1, 2, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Vehicle_Age',order_=True)
plt.title('Test Vehicle_Age Distribution')

In [ ]:
#Lets examine distribution of driving license possesion 
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [30, 10]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(2, 1, 1) 
single_categorical_prop_plot(data=train,variable='Region_Code',order_=True)
plt.title('Train Region_Code Distribution')

# histogram on right, example of too-small bin size
plt.subplot(2, 1, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Region_Code',order_=True)
plt.title('Test Region_Code Distribution')

Most of the of customers are from 28,8,46 region code.

In [ ]:
#Lets examine distribution of driving license possesion 
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [11, 5]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(1, 2, 1) 
single_categorical_prop_plot(data=train,variable='Previously_Insured',order_=True)
plt.title('Train Previously_Insured Distribution')

# histogram on right, example of too-small bin size
plt.subplot(1, 2, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Previously_Insured',order_=True)
plt.title('Test Previously_Insured Distribution')

In [ ]:
#Lets examine distribution of driving license possesion 
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [11, 5]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(1, 2, 1) 
single_categorical_prop_plot(data=train,variable='Vehicle_Damage',order_=True)
plt.title('Train Vehicle_Damage Distribution')

# histogram on right, example of too-small bin size
plt.subplot(1, 2, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Vehicle_Damage',order_=True)
plt.title('Test Vehicle_Damage Distribution')

In [ ]:
#Lets examine distribution of Policy_Sales_Channel 
# Resize the chart, and have two plots side-by-side
# set a larger figure size for subplots
plt.figure(figsize = [100, 10]) 
# histogram on left, example of too-large bin size
# 1 row, 2 cols, subplot 1
plt.subplot(2, 1, 1) 
single_categorical_prop_plot(data=train,variable='Policy_Sales_Channel',order_=True)
plt.title('Train Policy_Sales_Channel Distribution')

# histogram on right, example of too-small bin size
plt.subplot(2, 1, 2) # 1 row, 2 cols, subplot 2
single_categorical_prop_plot(data=test,variable='Policy_Sales_Channel',order_=True)
plt.title('Test Policy_Sales_Channel Distribution')

In [ ]:
train.head()

In [ ]:
# Resize the chart, and have two plots side-by-side
# Set a larger figure size for subplots
plt.figure(figsize = [15, 5]) 
sb.distplot(train['Age'])
plt.title('Distirbution Plot')

In [ ]:
train.Age.describe()

In [ ]:



#Define the figure size
plt.figure(figsize = [20, 5])

# histogram on left: full data
plt.subplot(1, 2, 1)
#bin_edges = np.arange(0, train['Annual_Premium'].max()+,)
sb.distplot(train['Annual_Premium'])
plt.title('Distirbution Plot')

# histogram on right: focus in on bulk of data < 6
plt.subplot(1, 2, 2)
bin_edges = np.arange(0, train['Annual_Premium'].max()+5000, 5000)
sb.distplot(train['Annual_Premium'], bins = bin_edges)
plt.xlim(0, 100000) # could also be called as plt.xlim((0, 6))
plt.title('Distirbution Plot')

In [ ]:
train.Annual_Premium.describe()

In [ ]:
# Resize the chart, and have two plots side-by-side
# Set a larger figure size for subplots
plt.figure(figsize = [15, 5]) 
sb.distplot(train['Vintage'])
plt.title('Distirbution Plot')

In [ ]:
train.Vintage.describe()

# Bivariate Exploration

In [ ]:
train.head()

In [ ]:
#Violin Plot
#categorical vs Numrical graph
plt.figure(figsize=[20,5])
base_color = sb.color_palette()[1]


# The "inner" argument represents the datapoints in the violin interior. 
# It can take any value from {“box”, “quartile”, “point”, “stick”, None}
# If "box", it draws a miniature boxplot. 
plt.subplot(1, 2, 1)
sb.violinplot(data=train, x='Response', y='Age', color=base_color, innner='quartile')
plt.title('Violin Plot')


plt.subplot(1, 2, 2)
sb.boxplot(data=train, y='Age', x='Response', color=base_color)
plt.title('Box Plot')